In [9]:
import torch
import time
import flair
from flair.models import TextClassifier
from flair.data import Sentence

sia = TextClassifier.load('en-sentiment')

def flair_prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    #print(score)
    if "POSITIVE" in str(score):
        return "pos"
    elif "NEGATIVE" in str(score):
        return "neg"
    else:
        return "neu"
    
def flair_prediction_value(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    return score.score

In [10]:
import pandas as pd
import os
folder_path = os.getcwd()
subdir_name = 'reviews'
new_subdirectory_path = os.path.join(folder_path, subdir_name)
if not os.path.exists(new_subdirectory_path):
    # Create the new subdirectory
        os.makedirs(new_subdirectory_path)
        print(f"Subdirectory '{subdir_name}' created successfully.")

app_name = "Impulse - Brain Training"
store_csv = "Impulse - Brain Training" + ".csv"
csv_file_path = os.path.join(new_subdirectory_path, store_csv)
mechanisms = ["has_leaderboard", "has_challenges", "has_points", "has_badges", "has_feedback", "has_social", "has_tracking",\
               "has_goals", "has_narrative", "has_collab"]
df = pd.read_csv(csv_file_path)


data_subdir_name = 'data_analysis'
new_subdirectory_path = os.path.join(folder_path, data_subdir_name)
if not os.path.exists(new_subdirectory_path):
    # Create the new subdirectory
        os.makedirs(new_subdirectory_path)
        print(f"Subdirectory '{data_subdir_name}' created successfully.")

In [11]:
df = df \
    .assign(has_leaderboard = lambda d: d['review'].str.contains('leaderboard', case=False))
df = df \
    .assign(has_challenges = lambda d: d['review'].str.contains('challenges', case=False))
df = df \
    .assign(has_points = lambda d: d['review'].str.contains('points', case=False))
df = df \
    .assign(has_badges = lambda d: d['review'].str.contains('badges', case=False))
df = df \
    .assign(has_feedback = lambda d: d['review'].str.contains('feedback|continuous', case=False))
df = df \
    .assign(has_social = lambda d: d['review'].str.contains('social|connection', case=False))
df = df \
    .assign(has_tracking = lambda d: d['review'].str.contains('tracking|best attempt|streak|score', case=False))
df = df \
    .assign(has_goals = lambda d: d['review'].str.contains('goals', case=False))
df = df \
    .assign(has_narrative = lambda d: d['review'].str.contains('narrative|story', case=False))
df = df \
    .assign(has_collab = lambda d: d['review'].str.contains('collaborating|collaboration|collaborate', case=False))



In [12]:
made_df = False

concat_df = pd.DataFrame()
store_csv = app_name + "_flair_data.csv"
csv_file_path = os.path.join(new_subdirectory_path, store_csv)

for y in mechanisms:
    print("The number of mentions of " ,y, "is " ,df[y].sum())
    if(df[y].sum()!= 0):
        f= df[y] #mask based on which mechansim we are aiming for
        df_modif = df[f]
        start = time.time()
        df_modif.loc[:, "flair_sentiment"] = df_modif["review"].apply(flair_prediction)
        end = time.time()
        print("Time was", end-start)
        df_modif["word_count"] = df_modif['review'].str.split().str.len()
        df_grouped = df_modif.groupby(["rating", "flair_sentiment"]).sum()
        df_grouped['Element type'] = y
        df_grouped.loc[0, 'Mentions'] = df[y].sum()
        if made_df == False:
            df_grouped.to_csv(csv_file_path)
            made_df = True
           # print("create csv")
        else:
            df_grouped.to_csv(csv_file_path, mode='a', header=not df_grouped.empty )
            #print("appending to csv")



The number of mentions of  has_leaderboard is  0
The number of mentions of  has_challenges is  93


C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\3882468432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modif.loc[:, "flair_sentiment"] = df_modif["review"].apply(flair_prediction)
C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\3882468432.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modif["word_count"] = df_modif['review'].str.split().str.len()


Time was 11.798348426818848
create csv
The number of mentions of  has_points is  3


C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\3882468432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modif.loc[:, "flair_sentiment"] = df_modif["review"].apply(flair_prediction)
C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\3882468432.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modif["word_count"] = df_modif['review'].str.split().str.len()


Time was 0.4289240837097168
appending to csv
The number of mentions of  has_badges is  0
The number of mentions of  has_feedback is  27


C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\3882468432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modif.loc[:, "flair_sentiment"] = df_modif["review"].apply(flair_prediction)
C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\3882468432.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modif["word_count"] = df_modif['review'].str.split().str.len()


Time was 5.821219205856323
appending to csv
The number of mentions of  has_social is  46


# Try to do analysis using VADER

In [5]:
import nltk
import time
nltk.download('vader_lexicon')
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
import operator

sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\iamre\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:

#df["sentiment"] = df["reviews.text"].apply(lambda x: max({k: sia.polarity_scores(x)[k] for k in good_keys}.items(), key=operator.itemgetter(1))[0])
df
made_vadercsv = False
store_csv = app_name + "_vader_data.csv"
csv_file_path = os.path.join(new_subdirectory_path, store_csv)


for y in mechanisms:
    print("The number of mentions of " ,y, "is " ,df[y].sum())
    if(df[y].sum()!= 0):
        f= df[y] #mask based on which mechansim we are aiming for
        df_vader = df[f]
        df_vader.loc[:, "word_count"] = df["review"].str.split().str.len()
        df_vader.loc[:, "vader_sentiment_score"] = df_vader.loc[:, "review"].apply(lambda x: sia.polarity_scores(x)["compound"])

        df_vader.loc[:, "vader_sentiment"] = np.select([df_vader["vader_sentiment_score"] < 0, df_vader["vader_sentiment_score"] == 0, df_vader["vader_sentiment_score"] > 0], ['neg', 'neu', 'pos'])
        df_vader.loc[:,'count'] = 1
        df_grouped = df_vader.groupby(["rating", "vader_sentiment"]).agg({'count': 'sum', 'vader_sentiment_score': 'mean'})
        df_grouped['Element type'] = y
        df_grouped.loc[0, 'Mentions'] = df[y].sum()
        #df.to_csv("strava_lb_sentiments.csv")

        if made_vadercsv == False:
            df_grouped.to_csv(csv_file_path)
            made_vadercsv = True
        else:
            df_grouped.to_csv(csv_file_path, mode='a', header=not df_grouped.empty )
        print("for ", y, "The vader analysis is")
        print(df_grouped)

The number of mentions of  has_leaderboard is  0
The number of mentions of  has_challenges is  93
for  has_challenges The vader analysis is
                        count  vader_sentiment_score    Element type  Mentions
rating vader_sentiment                                                        
1      pos                  1               0.990000  has_challenges        93
3      pos                  2               0.807900  has_challenges        93
4      neg                  1              -0.338900  has_challenges        93
       pos                  8               0.840775  has_challenges        93
5      neg                  2              -0.421150  has_challenges        93
       pos                 79               0.898652  has_challenges        93
The number of mentions of  has_points is  3
for  has_points The vader analysis is
                        count  vader_sentiment_score Element type  Mentions
rating vader_sentiment                                                

C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\619964034.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vader.loc[:, "word_count"] = df["review"].str.split().str.len()
C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\619964034.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vader.loc[:, "vader_sentiment_score"] = df_vader.loc[:, "review"].apply(lambda x: sia.polarity_scores(x)["compound"])
C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\619964034.py:16: SettingWithCopyWarning: 
A valu

for  has_social The vader analysis is
                        count  vader_sentiment_score Element type  Mentions
rating vader_sentiment                                                     
1      pos                  4               0.608450   has_social        46
3      pos                  1               0.788800   has_social        46
5      neg                  2              -0.171100   has_social        46
       pos                 39               0.833072   has_social        46
The number of mentions of  has_tracking is  82
for  has_tracking The vader analysis is
                        count  vader_sentiment_score  Element type  Mentions
rating vader_sentiment                                                      
1      neg                  2              -0.702050  has_tracking        82
       pos                  5               0.668760  has_tracking        82
2      neg                  1              -0.964100  has_tracking        82
       pos                  2     

C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\619964034.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vader.loc[:, "word_count"] = df["review"].str.split().str.len()
C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\619964034.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vader.loc[:, "vader_sentiment_score"] = df_vader.loc[:, "review"].apply(lambda x: sia.polarity_scores(x)["compound"])
C:\Users\iamre\AppData\Local\Temp\ipykernel_24324\619964034.py:16: SettingWithCopyWarning: 
A valu